## Hugging face

In [1]:
from transformers import pipeline
import os
import pypdf
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import gradio as gr
import torch


c:\Users\jakif\anaconda3\envs\modelo_1_NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Proving GPU is available

In [2]:
# Charge a pretrained model for question answering to use GPU
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased")


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Extract context through papers

In [3]:

def extract_text_from_papers(pdf_folder):
    texts = []
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, file)
            with open(pdf_path, "rb") as f:
                reader = pypdf.PdfReader(f)
                text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
                texts.append(text)
    return texts

# Extract text from papers
pdf_folder = "C:\PROYECTO_FINAL\dataset_papers"
papers_text = extract_text_from_papers(pdf_folder)

Implementation faiss

In [4]:

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Index texts with FAISS
db = FAISS.from_texts(papers_text, embeddings)
db.save_local("faiss_melanoma")

C:\Users\jakif\AppData\Local\Temp\ipykernel_5768\2331638720.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


## creation of the chatbot


In [5]:


qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")  


Device set to use cpu


In [6]:
def buscar_respuesta(pregunta):
    # Cargar la base de datos FAISS
    db = FAISS.load_local("faiss_melanoma", embeddings, allow_dangerous_deserialization=True)
    
    # Buscar en los papers
    docs = db.similarity_search(pregunta, k=3)  # Obtener los 3 textos más relevantes
    contexto = " ".join([doc.page_content for doc in docs])

    # Obtener la mejor respuesta con el modelo de QA
    respuesta = qa_pipeline(question=pregunta, context=contexto)
    
    return respuesta['answer']


In [7]:


def chatbot_melanoma(pregunta):
    respuesta = buscar_respuesta(pregunta)
    return respuesta

interfaz = gr.Interface(
    fn=chatbot_melanoma,
    inputs="text",
    outputs="text",
    title="Chatbot de Melanomas",
    description="Haz preguntas sobre melanomas basadas en papers científicos.",
)

interfaz.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
